In [1]:
import opt
#from distances import *
from sopt.lib_ot import *

In [2]:
# test the accuracy 
n_projections=50
n=200

X_projections=np.random.rand(n_projections,n)
Y_projections=np.random.rand(n_projections,n+20)
X_projections.sort(),Y_projections.sort()
Lambda_list=abs(np.random.rand(n_projections))

cost1,plan1=sliced_opt(X_projections,Y_projections,Lambda_list)
# cos2,plan2=your_function()
#error=np.linalg.norm(cost1-cost2)
#print(error)


In [ ]:
# test running time 
n_projections=50
n=20000

X_projections=np.random.rand(n_projections,n)
Y_projections=np.random.rand(n_projections,n)
X_projections.sort(),Y_projections.sort()
Lambda_list=abs(np.random.rand(n_projections))

%timeit sliced_opt(X_projections,Y_projections,Lambda_list)
%timeit opt.parallel_sliced_opt(X_projections,Y_projections,Lambda_list)
%timeit opt.sliced_opt(X_projections,Y_projections,Lambda_list)

1.47 s ± 62.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
#@nb.njit(['(float64)(float64[:,:],float64[:,:],float64,float64,float64,int64)'],fastmath=True,cache=True)
def sliced_ptlp_nb(signal1,signal2,beta,Lambda0,theta0, p=2.,n_projections=50):
    #Lambda_list is the \lambda in the paper and is the beta parameter in other functions 
    #beta_inv is 1/\beta in the paper and is the L parameter in other functions 
    x_a,x_b = np.expand_dims(np.linspace(0, 1, signal1.shape[0]), 1), np.expand_dims(np.linspace(0, 1, signal2.shape[0]), 1)
    D1,D2=~np.isnan(signal1.sum(1)),~np.isnan(signal2.sum(1))
    signal1, signal2 =signal1[D1], signal2[D2]
    x_a,x_b=x_a[D1],x_b[D2]
    K=x_a.shape[0]
    X_a,X_b=np.hstack(((1/beta)**(1/p)*x_a,signal1)),np.hstack(((1/beta)**(1/p)*x_b,signal2))
    n,d=X_a.shape
    projections = random_projections(d, n_projections,0)
    Lambda_list=np.full(n_projections,Lambda0)
    weights=np.abs(theta0.dot(projections.T))
    Lambda_list*=weights
    X_a_projections,X_b_projections=projections.dot(X_a.T),projections.dot(X_b.T) 
    X_a_projections.sort()
    X_b_projections.sort()
    opt_dist_list=sliced_opt(X_a_projections,X_b_projections,Lambda_list,p)
    sptlp_dist=np.sum(opt_dist_list)/n_projections*1/K
    return sptlp_dist

@nb.njit(cache=True)
def sliced_ptlp_nb2(signal1,signal2,beta,Lambda0,theta0, p=2.,n_projections=50):
    #Lambda_list is the \lambda in the paper and is the beta parameter in other functions 
    #beta_inv is 1/\beta in the paper and is the L parameter in other functions 
    x_a,x_b = np.expand_dims(np.linspace(0, 1, signal1.shape[0]), 1), np.expand_dims(np.linspace(0, 1, signal2.shape[0]), 1)
    D1,D2=~np.isnan(signal1.sum(1)),~np.isnan(signal2.sum(1))
    signal1, signal2 =signal1[D1], signal2[D2]
    x_a,x_b=x_a[D1],x_b[D2]
    K=x_a.shape[0]
    X_a,X_b=np.hstack(((1/beta)**(1/p)*x_a,signal1)),np.hstack(((1/beta)**(1/p)*x_b,signal2))
    n,d=X_a.shape
    projections = random_projections(d, n_projections,0)
    Lambda_list=np.full(n_projections,Lambda0)
    weights=np.abs(theta0.dot(projections.T))
    Lambda_list*=weights
    X_a_projections,X_b_projections=projections.dot(X_a.T),projections.dot(X_b.T) 
    X_a_projections.sort()
    X_b_projections.sort()
    opt_dist_list=sliced_opt(X_a_projections,X_b_projections,Lambda_list,p)
    sptlp_dist=np.sum(opt_dist_list)/n_projections*1/K
    return sptlp_dist

def sliced_ptlp_c(signal1,signal2,beta,Lambda0,theta0, p=2.,n_projections=50):
    #Lambda_list is the \lambda in the paper and is the beta parameter in other functions 
    #beta_inv is 1/\beta in the paper and is the L parameter in other functions 
    x_a,x_b = np.expand_dims(np.linspace(0, 1, signal1.shape[0]), 1), np.expand_dims(np.linspace(0, 1, signal2.shape[0]), 1)
    D1,D2=~np.isnan(signal1.sum(1)),~np.isnan(signal2.sum(1))
    signal1, signal2 =signal1[D1], signal2[D2]
    x_a,x_b=x_a[D1],x_b[D2]
    K=x_a.shape[0]
    X_a,X_b=np.hstack(((1/beta)**(1/p)*x_a,signal1)),np.hstack(((1/beta)**(1/p)*x_b,signal2))
    n,d=X_a.shape
    projections = random_projections(d, n_projections,0)
    Lambda_list=np.full(n_projections,Lambda0)
    weights=np.abs(theta0.dot(projections.T))
    Lambda_list*=weights
    X_a_projections,X_b_projections=projections.dot(X_a.T),projections.dot(X_b.T) 
    X_a_projections.sort()
    X_b_projections.sort()
    opt_dist_list,_=opt_c.sliced_opt(X_a_projections,X_b_projections,Lambda_list)
    sptlp_dist=np.sum(opt_dist_list)/n_projections*1/K
    return sptlp_dist

In [27]:
opt_c.solve

<function opt_c.PyCapsule.solve>

In [21]:
@nb.njit(fastmath=True,cache=True,parallel=True)
def sliced_opt1(X_projections,Y_projections,Lambda_list,p=2):
    #X_projections: n_projections* n shape, float64 
    #Y_projections: n_projections* m shape, float64 
    #Lambda_list: (n_projections,) shape, float64 
    #p 
    n_projections,n=X_projections.shape
    opt_cost_list=np.zeros(n_projections)
    opt_plan_list=np.zeros((n_projections,n))
    for epoch in range(n_projections):
        X_theta,Y_theta,Lambda=X_projections[epoch],Y_projections[epoch],Lambda_list[epoch]  
        #M=cost_matrix(X_theta,Y_theta,p)
        obj,phi,psi,piRow,piCol=opt_c.solve(X_theta,Y_theta,Lambda)
        opt_cost_list[epoch]=obj
        opt_plan_list[epoch]=piRow
    return opt_cost_list,opt_plan_list

@nb.njit(fastmath=True,cache=True,parallel=True)
def sliced_opt2(X_projections,Y_projections,Lambda_list,p=2):
    #X_projections: n_projections* n shape, float64 
    #Y_projections: n_projections* m shape, float64 
    #Lambda_list: (n_projections,) shape, float64 
    #p 
    n_projections,n=X_projections.shape
    opt_cost_list=np.zeros(n_projections)
    opt_plan_list=np.zeros((n_projections,n))
    for epoch in range(n_projections):
        X_theta,Y_theta,Lambda=X_projections[epoch],Y_projections[epoch],Lambda_list[epoch]  
        M=cost_matrix(X_theta,Y_theta,p)
        obj,phi,psi,piRow,piCol=solve_opt(M,Lambda)
        opt_cost_list[epoch]=obj
        opt_plan_list[epoch]=piRow
    return opt_cost_list,opt_plan_list

In [22]:
X_projections=np.random.rand(1000,20)
Y_projections=np.random.rand(1000,50)
X_projections.sort()
Y_projections.sort()
Lambda_list=abs(np.random.rand(1000))
sliced_opt1(X_projections,Y_projections,Lambda_list,p=2)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'solve' of type Module(<module 'opt_c' from '/home/baly/projects/PTLP/PTLP/opt_c.so'>)

File "../../../../../tmp/ipykernel_3550270/681616093.py", line 13:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_3550270/681616093.py (13)

File "../../../../../tmp/ipykernel_3550270/681616093.py", line 13:
<source missing, REPL/exec in use?>


In [ ]:
signal1=np.random.rand(5000,1)
signal2=np.random.rand(5000,1)
beta=10.0
Lambda0=0.2
theta0=np.array([.0,1.0])
%timeit sliced_ptlp_nb(signal1,signal2,beta,Lambda0,theta0, p=2.,n_projections=50)
%timeit sliced_ptlp_nb2(signal1,signal2,beta,Lambda0,theta0, p=2.,n_projections=1)
%timeit sliced_ptlp_c(signal1,signal2,beta,Lambda0,theta0, p=2.,n_projections=1)

In [30]:
for i in range(10):
    X_projections=np.random.rand(10,1000)
    Y_projections=np.random.rand(10,1000)
    Lambda_list=abs(np.random.rand(10))
    X_projections.sort()
    Y_projections.sort()
    print('done')
    obj1,plan1=sliced_opt_C(X_projections,Y_projections,Lambda_list)
    obj2,plan2=opt.sliced_opt(X_projections,Y_projections,Lambda_list)
    error=np.linalg.norm(obj1-obj2)
    print(error)

done


NameError: name 'opt' is not defined

In [6]:
X_projections=np.random.rand(10,1000)
Y_projections=np.random.rand(10,1000)
Lambda_list=abs(np.random.rand(10))
X_projections.sort()
Y_projections.sort()

In [7]:
opt.sliced_opt(X_projections, Y_projections, Lambda_list)

(array([0.27623354, 0.07774271, 0.54588066, 0.67116766, 0.22766288,
        0.25776601, 0.23595076, 0.09726555, 0.1566605 , 0.11533289]),
 array([[  0,   1,   2, ..., 997, 998, 999],
        [  0,   1,   2, ..., 997, 998, 999],
        [  0,   1,   2, ..., 997, 998, 999],
        ...,
        [  0,   1,   2, ..., 997, 998, 999],
        [  0,   1,   2, ..., 997, 998, 999],
        [  0,   1,   2, ..., 997, 998, 999]], dtype=int32))